In [1]:
# Importing the required libraries.
import numpy as np
import pandas as pd
import pickle, zlib
from random import sample
import scipy.cluster.hierarchy as sch
from gensim.models.doc2vec import Doc2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import single, fcluster
from scipy.spatial.distance import pdist
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import single, cophenet
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.neighbors import NearestCentroid
from collections import Counter
import sys
import math
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))  
from nltk.tokenize import RegexpTokenizer
from collections import Counter
import seaborn as sns
sys.setrecursionlimit(5000)
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def read_index(res_ids,vectors,ids):
    #with open(filename,'r') as f:
    #    text = f.read()
    #res_ids = text.split('\n')
    result =[]
    ids = list(ids)
    indd = []
    for i in range(len(res_ids)):
        if res_ids[i] in ids:
            result.append(vectors[ids.index(res_ids[i])])
            indd.append(ids.index(res_ids[i]))
        else:
            print(i,"Id not found")
    return indd,result

In [3]:
def global_centroid(data):
    return np.median(np.array(data),axis=0)


In [4]:
def get_rank(data,global_centroid):
    ranks = []
    for d in data:
        ranks.append((cosine_similarity(global_centroid.reshape(1,-1),d.reshape(1,-1))[0][0]+1)/2) #cos_sim(row1, row2)- minx)/(maxx-minx)
    return ranks
    

In [5]:
def givetfidf(titles,n=10):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(titles)
    feature_array = np.array(vectorizer.get_feature_names())
    tfidf_sorting = np.argsort(X.toarray()).flatten()[::-1]
    top_n = feature_array[tfidf_sorting][:n]
    last_n = feature_array[tfidf_sorting][-n:]
    return top_n, last_n


In [6]:
df = pd.read_excel('Files/ADI.xlsx')
census = list(df['census_code'])
dist_names = list(df['name'])
distt = {}
for i,j in zip(census,dist_names):
    distt[i]=j

In [8]:
import pandas as pd
datasets = ['dataset_agriculture', 'dataset_development', 'dataset_environment', 'dataset_industrialization', 'dataset_lifestyle']
model_names = ['model_agriculture', 'model_development', 'model_environment', 'model_industrialization', 'model_lifestyle']
final_res_all = []
for dataset, model in zip(datasets,model_names):
    dataset_name = dataset
    # Printing the collection name.
    collection_name = dataset[8:]
    print('\nCollection:',collection_name.capitalize())
    # Loading the dataset and the model from the drive.
    file = open('Datasets/'+dataset, 'rb')
    dataset = pickle.loads(zlib.decompress(pickle.load(file)))
    file.close()
    model = Doc2Vec.load('Models/'+model)

    # Collecting the article_ids, and corresponding article_vectors for each class.
    temp_ids = [[] for _ in range(9)]
    temp_titles = [[] for _ in range(9)]
    temp_vectors = [[] for _ in range(9)]
    temp_datasets = [[] for _ in range(9)]
    temp_dists = [[] for _ in range(9)]
    for i in dataset:
        if i[6]=='Unemp' and i[-1]=='Average':
            if i[0] not in temp_ids[0]:
                temp_ids[0].append(i[0])
                temp_titles[0].append(i[1])
                temp_dists[0].append(i[-5])
                temp_vectors[0].append(model.docvecs[i[0]])
                temp_datasets[0].append([i[0],model.docvecs[i[0]]])
        if i[6]=='Unemp' and i[-1]=='Slow':
            if i[0] not in temp_ids[1]:
                temp_ids[1].append(i[0])
                temp_dists[1].append(i[-5])
                temp_titles[1].append(i[1])
                temp_vectors[1].append(model.docvecs[i[0]])
                temp_datasets[1].append([i[0],model.docvecs[i[0]]])
        if i[6]=='Unemp' and i[-1]=='Fast':
            if i[0] not in temp_ids[2]:
                temp_ids[2].append(i[0])
                temp_dists[2].append(i[-5])
                temp_titles[2].append(i[1])
                temp_vectors[2].append(model.docvecs[i[0]])
                temp_datasets[2].append([i[0],model.docvecs[i[0]]])
        if i[6]=='Agri' and i[-1]=='Average':
            if i[0] not in temp_ids[3]:
                temp_ids[3].append(i[0])
                temp_dists[3].append(i[-5])
                temp_titles[3].append(i[1])
                temp_vectors[3].append(model.docvecs[i[0]])
                temp_datasets[3].append([i[0],model.docvecs[i[0]]])
        if i[6]=='Agri' and i[-1]=='Slow':
            if i[0] not in temp_ids[4]:
                temp_ids[4].append(i[0])
                temp_dists[4].append(i[-5])
                temp_titles[4].append(i[1])
                temp_vectors[4].append(model.docvecs[i[0]])
                temp_datasets[4].append([i[0],model.docvecs[i[0]]])
        if i[6]=='Agri' and i[-1]=='Fast':
            if i[0] not in temp_ids[5]:
                temp_ids[5].append(i[0])
                temp_dists[5].append(i[-5])
                temp_titles[5].append(i[1])
                temp_vectors[5].append(model.docvecs[i[0]])
                temp_datasets[5].append([i[0],model.docvecs[i[0]]])
        if i[6]=='Non Agri' and i[-1]=='Average':
            if i[0] not in temp_ids[6]:
                temp_ids[6].append(i[0])
                temp_dists[6].append(i[-5])
                temp_titles[6].append(i[1])
                temp_vectors[6].append(model.docvecs[i[0]])
                temp_datasets[6].append([i[0],model.docvecs[i[0]]])
        if i[6]=='Non Agri' and i[-1]=='Slow':
            if i[0] not in temp_ids[7]:
                temp_ids[7].append(i[0])
                temp_dists[7].append(i[-5])
                temp_titles[7].append(i[1])
                temp_vectors[7].append(model.docvecs[i[0]])
                temp_datasets[7].append([i[0],model.docvecs[i[0]]])
        if i[6]=='Non Agri' and i[-1]=='Fast':
            if i[0] not in temp_ids[8]:
                temp_ids[8].append(i[0])
                temp_dists[8].append(i[-5])
                temp_titles[8].append(i[1])
                temp_vectors[8].append(model.docvecs[i[0]])
                temp_datasets[8].append([i[0],model.docvecs[i[0]]])
    names = ['unemp_avg_id','unemp_slow_id','unemp_fast_id','agri_avg_id','agri_slow_id','agri_fast_id','non_agri_avg_id','non_agri_slow_id','non_agri_fast_id']
    df = pd.read_excel('DocTag2Vec/'+collection_name+'_ids.xlsx')
    all_val = []
    dff= pd.DataFrame()
    for m in [0,3,6]:
        indd1,vec1 = read_index(list(df[names[m+1]]),temp_vectors[m+1],temp_ids[m+1])
        indd2,vec2 = read_index(list(df[names[m+2]]),temp_vectors[m+2],temp_ids[m+2])
        global1 = global_centroid(vec1)
        global2 = global_centroid(vec2)
        rankA1 = get_rank(vec1,global1)
        rankB1 = get_rank(vec1,global2)
        rankA2 = get_rank(vec2,global2)
        rankB2 = get_rank(vec2,global1)
        score1 = list(np.argsort([i-j for i,j in zip(rankA1,rankB1)]))
        score2 = list(np.argsort([i-j for i,j in zip(rankA2,rankB2)]))
        l1 = []
        v1 =[]
        print("\nHighest similarity in ",names[m+1])
        for i in score1[-30:]:
            v = temp_dists[m+1][indd1[i]]
            if v in distt:
                v = distt[v]
            l1.append(temp_titles[m+1][indd1[i]])
            v1.append(v)
            print(temp_titles[m+1][indd1[i]],temp_dists[m+1][indd1[i]],v)
        dff[names[m+1]+'_'+'high']=l1
        dff[names[m+1]+'_'+'high_dist']=v1
        l1= []
        v1=[]
        print("\nHighest similarity in ",names[m+2])
        for i in score2[-30:]:
            v = temp_dists[m+2][indd2[i]]
            if v in distt:
                v = distt[v]
            l1.append(temp_titles[m+2][indd2[i]])
            v1.append(v)
            print(temp_titles[m+2][indd2[i]],temp_dists[m+2][indd2[i]],v)   
        dff[names[m+2]+'_'+'high']=l1
        dff[names[m+2]+'_'+'high_dist']=v1
    #dff.to_excel(collection_name+'high.xlsx',index=False)


Collection: Agriculture


Lowest similarity in  unemp_stag_id

Highest similarity in  unemp_stag_id
Nitish woos investors in Japan 230 Patna
Govt to seek $750-m WB loan for Kosi rehab 230 Patna
Reach rural areas or wind up, state finance minister warns banks 230 Patna
Tenders for Polavaram project soon 544 Visakhapatnam
New chief for SSB 230 Patna
Rahul goes rural,so does Congress 230 Patna
Toppers aim to follow footsteps of Abdul Kalam 175 Allahabad 
Mayawati using bullets to lay a new tradition of development, say rivals 146 Agra
Lalu: Flood-hit lucky to get Gangajal at doorstep 230 Patna
JD(U) to back Jaswant Singh for VP 230 Patna
Nimesh report to be tabled at Vidhan Sabha monsoon session 177 Faizabad
Threat triggers exodus of Mahadalits from Bihar village, four sent to jail 236 Gaya
Populist promises not what state needs 230 Patna
CM promises fast execution of key policies 146 Agra
Aamir Khan gets relief from Patna high court 230 Patna
Yogi govt proposes ‘Make in UP’ department f



Lowest similarity in  unemp_stag_id

Highest similarity in  unemp_stag_id
Scheme page on Google  236 Gaya
Unfazed Jitan Ram Manjhi calls cabinet meet again 230 Patna
CM on firm ground amid stirs 360 Hazaribagh
Minister bats for PMC solution 230 Patna
West Bengal gives Rs 2 lakh each to families of labourers killed in UP blast 175 Allahabad 
Review 236 Gaya
Teachers 236 Gaya
Govt Course Tailored for Visually Impaired Students in MP Fails to Teach Practicals 430 Rewa
Bhadrak labours to utilize central funds 378 Bhadrak
CM Nitish Kumar opens registration & counselling centre for youths 230 Patna
Doba damages 364 Ranchi
CM Das prods fast action 351 Godda
Status cry to reach NDC meet 230 Patna
State okays funds for enhanced salary to varsity teachers 230 Patna
Mukhiyas threaten stir 358 Palamu
'People's budget' amid Opp. boycott 362 Dumka
Prod on projects 351 Godda
Power bill reward scheme a non-starter 362 Dumka
‘Rural’ ire at BJP leader’s home 230 Patna
Leader held 230 Patna
4 rebels ar


Collection: Environment


Lowest similarity in  unemp_stag_id

Highest similarity in  unemp_stag_id
District administration to plant more than 12 lakh saplings 175 Allahabad 
Headless torsos found 230 Patna
Indira Gandhi for the millennials 175 Allahabad 
Smaller parties turn to Naxalites for poll success 196 Chandauli
Quick fix eludes agitation 230 Patna
Poplar power for rural economy 230 Patna
Voters want MLAs to redress bad roads, traffic 175 Allahabad 
Zirakpur nightclub booked second time in a month 43 Firozpur
Kidnapped youth recovered in Munger, 4 held 230 Patna
Rabri to resume Chhath puja after marriage of sons 230 Patna
Maoists cornered in Magadh division 239 Jehanabad 
Mandate was to serve people, not to amass wealth, says CM Nitish Kumar 230 Patna
Nitish, Patil pitch for quality education 230 Patna
Maoist leader gives up arms 358 Palamu
Akhilesh Yadav pledges greener Lucknow 46 Bathinda
Supreme Court ban on crackers goes up in smoke 230 Patna
The Matua factor in Bengal poli



Lowest similarity in  unemp_stag_id

Highest similarity in  unemp_stag_id
Come summer, collegians go for internship programme 230 Patna
Note ban: Budget hotels see low biz during peak season 146 Agra
Virtual hits for real fame 364 Ranchi
Temple WiFi misuse: Agra priest booked 146 Agra
Patna University to appoint experts as adjunct faculty members 230 Patna
District administration lends a helping hand to poverty-struck marathon winner 175 Allahabad 
4Ds a major hurdle for kids' growth 175 Allahabad 
‘Controversies can affect Kerala’s tourism prospects’ 146 Agra
Incarcerated sharp-shooter made 188 calls to girlfriend 146 Agra
Tech, telecom firms roll out freebies for Diwali 230 Patna
Scheme pat 230 Patna
Better range to promote shooters 229 Nalanda
Govt taking steps to boost dairy industry 229 Nalanda
Airtel blames lack of mobile towers for frequent call drops 74 Karnal
PMC fails to keep Patna hangouts clean 230 Patna
Rural job days to double 230 Patna
Crackdown against sand mining maf

Collection: Lifestyle


Lowest similarity in  unemp_stag_id

Highest similarity in  unemp_stag_id
Bihar celebrates foundation day with government, Opposition sparring over gains from prohibition 230 Patna
Internet services suspended in 7 districts 216 Muzaffarpur
Euphoria to hit IIT 230 Patna
Government lacks work culture: HC 230 Patna
Seashore Group’s office in Cuttack seized 384 Anugul
'Economic reasons for migration in W UP' 175 Allahabad 
Vishwaroopam released in U.P. amid tight security 146 Agra
Catch them young for preventing heart ailments 230 Patna
BJP's Kuldeep Singh Sengar Sent to 7-Day CBI Custody in Unnao Rape Case 156 Unnao
Cycle pump surgery: 2 doctors suspended 384 Anugul
Fresher function for UG and PG students at SHIATS 175 Allahabad 
Gaya City DSP served show-cause notice, SHO suspended 230 Patna
Delhi smog: Rise in asthma, allergy, breathlessness cases 230 Patna
Bihar govt to rope in NGOs for implementing liquor ban 230 Patna
‘Happy’ news for Didi 230 Patna
Shootouts 